In [ ]:
import numpy as np

def align(seq1, seq2, match=1, gap_penalty=1, mismatch_penalty=1):
    m = len(seq1)
    n = len(seq2)
    
    # Score matrix H, initialized to 0
    H = np.zeros((m + 1, n + 1), dtype=int)
    
    best_score = 0
    best_pos = (0, 0)  # (i, j) where best_score occurs
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # score from diagonal (match/mismatch)
            if seq1[i - 1] == seq2[j - 1]:
                score_diag = H[i - 1, j - 1] + match
            else:
                score_diag = H[i - 1, j - 1] - mismatch_penalty
            
            # score from up (gap in seq2)
            score_up   = H[i - 1, j] - gap_penalty
            # score from left (gap in seq1)
            score_left = H[i, j - 1] - gap_penalty
            
            # Smith–Waterman: local alignment -> no negative scores
            H[i, j] = max(0, score_diag, score_up, score_left)
            
            # track global maximum
            if H[i, j] > best_score:
                best_score = H[i, j]
                best_pos = (i, j)
    
    aligned1 = []
    aligned2 = []
    i, j = best_pos
    
    # Do not store parent pointers
    # Recompute which predecessor could have produced H[i, j]
    while i > 0 and j > 0 and H[i, j] > 0:
        score_current = H[i, j]
        
        # recompute predecessors
        if seq1[i - 1] == seq2[j - 1]:
            score_diag = H[i - 1, j - 1] + match
        else:
            score_diag = H[i - 1, j - 1] - mismatch_penalty
        
        score_up   = H[i - 1, j] - gap_penalty
        score_left = H[i, j - 1] - gap_penalty
        
        # prefer diagonal when possible, then up, then left 
        if score_current == score_diag:
            aligned1.append(seq1[i - 1])
            aligned2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif score_current == score_up:
            aligned1.append(seq1[i - 1])
            aligned2.append('-')
            i -= 1
        elif score_current == score_left:
            aligned1.append('-')
            aligned2.append(seq2[j - 1])
            j -= 1
        else:
            # if no predecessor matches, stop
            break
    
    aligned1 = ''.join(reversed(aligned1))
    aligned2 = ''.join(reversed(aligned2))
    
    return aligned1, aligned2, int(best_score)

In [ ]:
# given tests
seq1, seq2, score = align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', gap_penalty=2)
print(seq1)
print(seq2)
print(score)

seq3, seq4, score2 = align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac')
print(seq3)
print(seq4)
print(score2)

gcatcga
gcatcga
7
agacccta-cgt-gac
aga-cctagcatcgac
8


In [ ]:
# additional tests 
# test 1: identical strings with default parameters
align("ACGT", "ACGT")  # default match=1, gap_penalty=1, mismatch_penalty=1
# expected output: ("ACGT", "ACGT", 4)

('ACGT', 'ACGT', 4)

In [8]:
# test 2: completely different strings with high mismatch penalty
align("AAAA", "TTTT", match=2, mismatch_penalty=3, gap_penalty=2)
# expected output: ("", "", 0)

('', '', 0)

In [ ]:
# test 3: symmetry score test
sc_ab = align("GATTACA", "GCATGCU")[2]
sc_ba = align("GCATGCU", "GATTACA")[2]
assert sc_ab == sc_ba, "Alignment scores should be symmetric"

In [ ]:
# test 4: gap penalty effect test
low_gap = align("ACGTACGT", "ACGACGT", gap_penalty=1)
high_gap = align("ACGTACGT", "ACGACGT", gap_penalty=4)

low_gap, high_gap

(('ACGTACGT', 'ACG-ACGT', 6), ('ACGT', 'ACGT', 4))